In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
p1 = Path.cwd() / 'data'

In [4]:
flights = pd.read_csv(p1 / 'flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [7]:
(flights
 .groupby('AIRLINE')[['ARR_DELAY']]
 .mean()
)

,ARR_DELAY
AIRLINE,
AA,5.542661
AS,-0.833333
B6,8.692593
DL,0.339691
EV,7.034580
F9,13.630651
HA,4.972973
MQ,6.860591
NK,18.436070


In [8]:
(flights
 .groupby(['AIRLINE', 'WEEKDAY'])['CANCELLED']
 .sum()
)

AIRLINE  WEEKDAY
AA       1          41
         2           9
         3          16
         4          20
         5          18
                    ..
WN       3          18
         4          10
         5           7
         6          10
         7           7
Name: CANCELLED, Length: 98, dtype: int64

In [10]:
(flights
 .groupby(['AIRLINE', 'WEEKDAY'])[['CANCELLED', 'DIVERTED']]
 .agg(['sum', 'mean'])
)

CANCELLED           DIVERTED          
                      sum      mean      sum      mean
AIRLINE WEEKDAY                                       
AA      1              41  0.032106        6  0.004699
        2               9  0.007341        2  0.001631
        3              16  0.011949        2  0.001494
        4              20  0.015004        5  0.003751
        5              18  0.014151        1  0.000786
...                   ...       ...      ...       ...
WN      3              18  0.014118        2  0.001569
        4              10  0.007911        4  0.003165
        5               7  0.005828        0  0.000000
        6              10  0.010132        3  0.003040
        7               7  0.006066        3  0.002600

[98 rows x 4 columns]

In [12]:
(flights
 .groupby(['ORG_AIR', 'DEST_AIR'])
 .agg({'CANCELLED':['sum', 'mean', 'size'], 'AIR_TIME':['mean', 'var']})
)

CANCELLED                   AIR_TIME            
                       sum      mean size        mean         var
ORG_AIR DEST_AIR                                                 
ATL     ABE              0  0.000000   31   96.387097   45.778495
        ABQ              0  0.000000   16  170.500000   87.866667
        ABY              0  0.000000   19   28.578947    6.590643
        ACY              0  0.000000    6   91.333333   11.466667
        AEX              0  0.000000   40   78.725000   47.332692
...                    ...       ...  ...         ...         ...
SFO     SNA              4  0.032787  122   64.059322   11.338331
        STL              0  0.000000   20  198.900000  101.042105
        SUN              0  0.000000   10   78.000000   25.777778
        TUS              0  0.000000   20  100.200000   35.221053
        XNA              0  0.000000    2  173.500000    0.500000

[1130 rows x 5 columns]

In [15]:
(flights
 .groupby(['ORG_AIR', 'DEST_AIR'])
 .agg(sum_cancelled=pd.NamedAgg(column='CANCELLED', aggfunc='sum'),
      mean_cancelled=pd.NamedAgg(column='CANCELLED', aggfunc='mean'),
      mean_air_time=pd.NamedAgg(column='AIR_TIME', aggfunc='mean'))
)

sum_cancelled  mean_cancelled  mean_air_time
ORG_AIR DEST_AIR                                              
ATL     ABE                   0        0.000000      96.387097
        ABQ                   0        0.000000     170.500000
        ABY                   0        0.000000      28.578947
        ACY                   0        0.000000      91.333333
        AEX                   0        0.000000      78.725000
...                         ...             ...            ...
SFO     SNA                   4        0.032787      64.059322
        STL                   0        0.000000     198.900000
        SUN                   0        0.000000      78.000000
        TUS                   0        0.000000     100.200000
        XNA                   0        0.000000     173.500000

[1130 rows x 3 columns]

In [16]:
res = (flights
 .groupby(['ORG_AIR', 'DEST_AIR'])
 .agg({'CANCELLED':['sum', 'mean', 'size'], 'AIR_TIME':['mean', 'var']})
)
res

CANCELLED                   AIR_TIME            
                       sum      mean size        mean         var
ORG_AIR DEST_AIR                                                 
ATL     ABE              0  0.000000   31   96.387097   45.778495
        ABQ              0  0.000000   16  170.500000   87.866667
        ABY              0  0.000000   19   28.578947    6.590643
        ACY              0  0.000000    6   91.333333   11.466667
        AEX              0  0.000000   40   78.725000   47.332692
...                    ...       ...  ...         ...         ...
SFO     SNA              4  0.032787  122   64.059322   11.338331
        STL              0  0.000000   20  198.900000  101.042105
        SUN              0  0.000000   10   78.000000   25.777778
        TUS              0  0.000000   20  100.200000   35.221053
        XNA              0  0.000000    2  173.500000    0.500000

[1130 rows x 5 columns]

In [23]:
res.columns = [f'{b}_{a}' for a, b in res.columns.to_flat_index()]

In [24]:
res

sum_CANCELLED  mean_CANCELLED  size_CANCELLED  \
ORG_AIR DEST_AIR                                                  
ATL     ABE                   0        0.000000              31   
        ABQ                   0        0.000000              16   
        ABY                   0        0.000000              19   
        ACY                   0        0.000000               6   
        AEX                   0        0.000000              40   
...                         ...             ...             ...   
SFO     SNA                   4        0.032787             122   
        STL                   0        0.000000              20   
        SUN                   0        0.000000              10   
        TUS                   0        0.000000              20   
        XNA                   0        0.000000               2   

                  mean_AIR_TIME  var_AIR_TIME  
ORG_AIR DEST_AIR                               
ATL     ABE           96.387097     45.778495  
        ABQ          170.500000     87.866667  
        ABY           28.578947      6.590643  
        ACY           91.333333     11.466667  
        AEX           78.725000     47.332692  
...                         ...           ...  
SFO     SNA           64.059322     11.338331  
        STL          198.900000    101.042105  
        SUN           78.000000     25.777778  
        TUS          100.200000     35.221053  
        XNA          173.500000      0.500000  

[1130 rows x 5 columns]

In [39]:
def flatten_cols(df):
    df_copy = df.copy()
    df_copy.columns = [f'{b}_{a}' for a, b in df.columns.to_flat_index()]
    return df_copy

In [26]:
(flights
 .groupby(['ORG_AIR', 'DEST_AIR'])
 .agg({'CANCELLED':['sum', 'mean', 'size'], 'AIR_TIME':['mean', 'var']})
 .pipe(flatten_cols)
)

sum_CANCELLED  mean_CANCELLED  size_CANCELLED  \
ORG_AIR DEST_AIR                                                  
ATL     ABE                   0        0.000000              31   
        ABQ                   0        0.000000              16   
        ABY                   0        0.000000              19   
        ACY                   0        0.000000               6   
        AEX                   0        0.000000              40   
...                         ...             ...             ...   
SFO     SNA                   4        0.032787             122   
        STL                   0        0.000000              20   
        SUN                   0        0.000000              10   
        TUS                   0        0.000000              20   
        XNA                   0        0.000000               2   

                  mean_AIR_TIME  var_AIR_TIME  
ORG_AIR DEST_AIR                               
ATL     ABE           96.387097     45.778495  
        ABQ          170.500000     87.866667  
        ABY           28.578947      6.590643  
        ACY           91.333333     11.466667  
        AEX           78.725000     47.332692  
...                         ...           ...  
SFO     SNA           64.059322     11.338331  
        STL          198.900000    101.042105  
        SUN           78.000000     25.777778  
        TUS          100.200000     35.221053  
        XNA          173.500000      0.500000  

[1130 rows x 5 columns]

In [40]:
air_info = (flights
            .groupby(['AIRLINE', 'WEEKDAY'])
            .agg({'DIST':['sum', 'mean'], 'ARR_DELAY':['min', 'max']})
            .astype('int')
)
air_info

DIST       ARR_DELAY     
                     sum  mean       min  max
AIRLINE WEEKDAY                              
AA      1        1455386  1139       -60  551
        2        1358256  1107       -52  725
        3        1496665  1117       -45  473
        4        1452394  1089       -46  349
        5        1427749  1122       -41  732
...                  ...   ...       ...  ...
WN      3         997213   782       -38  262
        4        1024854   810       -52  284
        5         981036   816       -44  244
        6         823946   834       -41  290
        7         945679   819       -45  261

[98 rows x 4 columns]

In [41]:
air_info.columns.get_level_values(0), air_info.columns.get_level_values(1)

(Index(['DIST', 'DIST', 'ARR_DELAY', 'ARR_DELAY'], dtype='object'),
 Index(['sum', 'mean', 'min', 'max'], dtype='object'))

In [44]:
air_info.pipe(flatten_cols).reset_index()

,AIRLINE,WEEKDAY,sum_DIST,mean_DIST,min_ARR_DELAY,max_ARR_DELAY
0,AA,1,1455386,1139,-60,551
1,AA,2,1358256,1107,-52,725
2,AA,3,1496665,1117,-45,473
3,AA,4,1452394,1089,-46,349
4,AA,5,1427749,1122,-41,732
...,...,...,...,...,...,...
93,WN,3,997213,782,-38,262
94,WN,4,1024854,810,-52,284
95,WN,5,981036,816,-44,244
96,WN,6,823946,834,-41,290


In [45]:
college = pd.read_csv(p1 / 'college.csv')

In [46]:
(college
 .groupby('STABBR')['UGDS']
 .agg(['mean', 'std'])
 .round(0)
)

,mean,std
STABBR,,
AK,2493.0,4052.0
AL,2790.0,4658.0
AR,1644.0,3143.0
AS,1276.0,NaN
AZ,4130.0,14894.0
CA,3518.0,6709.0
CO,2325.0,4670.0
CT,1874.0,2871.0
DC,2645.0,3225.0


In [47]:
def max_dev(s):
    std_score = (s - s.mean()) / s.std()
    return std_score.abs().max()

In [49]:
(college
 .groupby('STABBR')['UGDS']
 .agg(max_dev)
 .round(1)
)

STABBR
AK     2.6
AL     5.8
AR     6.3
AS     NaN
AZ     9.9
CA     6.1
CO     5.0
CT     5.6
DC     2.4
DE     3.5
FL     8.4
FM     NaN
GA     5.4
GU     1.0
HI     3.8
IA     6.5
ID     4.5
IL     7.3
IN     9.1
KS     4.9
KY     5.2
LA     6.5
MA     6.1
MD     5.3
ME     4.0
MH     NaN
MI     6.7
MN     7.8
MO     7.2
MP     NaN
MS     4.0
MT     3.9
NC     4.9
ND     3.5
NE     5.0
NH     5.3
NJ     7.1
NM     4.5
NV     4.7
NY     8.2
OH    10.3
OK     5.9
OR     5.3
PA    10.1
PR     6.0
PW     NaN
RI     2.9
SC     6.0
SD     4.2
TN     6.0
TX     7.7
UT     5.1
VA     7.0
VI     NaN
VT     3.8
WA     6.6
WI     5.8
WV     7.2
WY     2.8
Name: UGDS, dtype: float64

In [57]:
def pct_bet(s, low, high):
    return round(s.between(low, high).mean() * 100, 2)

In [59]:
(college
 .groupby(['STABBR', 'RELAFFIL'])['UGDS']
 .agg(pct_bet, low=1000, high=10000)
)

STABBR  RELAFFIL
AK      0           42.86
        1            0.00
AL      0           45.83
        1           37.50
AR      0           39.71
                    ...  
WI      0           31.03
        1           44.00
WV      0           29.23
        1           37.50
WY      0           72.73
Name: UGDS, Length: 112, dtype: float64

In [60]:
def bet_n_m(n, m):
    def wrapper(ser):
        return pct_bet(ser, n, m)
    wrapper.__name__ = f'between_{n}_{m}'
    return wrapper

In [61]:
(college
 .groupby(['STABBR', 'RELAFFIL'])['UGDS']
 .agg(bet_n_m(1000, 10000))
)

STABBR  RELAFFIL
AK      0           42.86
        1            0.00
AL      0           45.83
        1           37.50
AR      0           39.71
                    ...  
WI      0           31.03
        1           44.00
WV      0           29.23
        1           37.50
WY      0           72.73
Name: UGDS, Length: 112, dtype: float64

In [63]:
grouped = college.groupby(['STABBR', 'RELAFFIL'])
grouped

In [65]:
grouped.ngroups, grouped.groups

(112,
 {('AK', 0): [60, 62, 63, 65, 66, 67, 5171], ('AK', 1): [61, 64, 5417], ('AL', 0): [0, 1, 3, 4, 5, 6, 7, 8, 9, 11, 13, 14, 15, 17, 18, 19, 20, 21, 22, 25, 26, 27, 28, 29, 31, 32, 33, 34, 37, 38, 39, 40, 42, 44, 45, 48, 49, 50, 52, 55, 56, 57, 58, 59, 4307, 4415, 4483, 4484, 4851, 4872, 4986, 4987, 5202, 5296, 5523, 5564, 5879, 5885, 5890, 5924, 5997, 6013, 6062, 6124, 6126, 6134, 6139, 6600, 6685, 6799, 6894, 7079], ('AL', 1): [2, 10, 12, 16, 23, 24, 30, 35, 36, 41, 46, 47, 51, 53, 54, 5818, 6753, 7061, 7164, 7165, 7166, 7167, 7168, 7495], ('AR', 0): [128, 129, 130, 132, 133, 135, 136, 137, 138, 139, 140, 141, 142, 143, 145, 146, 148, 150, 151, 152, 153, 154, 155, 157, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 172, 173, 175, 177, 178, 179, 180, 181, 182, 183, 184, 187, 188, 189, 190, 191, 4443, 4834, 4999, 5250, 5315, 5591, 5632, 5868, 6184, 6234, 6364, 6512, 6628, 6633, 6891, 7117], ('AR', 1): [131, 134, 144, 147, 149, 156, 158, 171, 174, 176, 185, 186, 4485, 5

In [67]:
grouped.get_group(('FL', 1)).head()

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
712,The Baptist College of Florida,Graceville,FL,0.0,0.0,0.0,1,545.0,465.0,0.0,...,0.0308,0.0000,0.0507,0.2291,1,0.5878,0.5602,0.3531,30800,20052
713,Barry University,Miami,FL,0.0,0.0,0.0,1,470.0,462.0,0.0,...,0.0164,0.0741,0.0841,0.1518,1,0.5045,0.6733,0.4361,44100,28250
714,Gooding Institute of Nurse Anesthesia,Panama City,FL,0.0,0.0,0.0,1,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,PrivacySuppressed
715,Bethune-Cookman University,Daytona Beach,FL,1.0,0.0,0.0,1,405.0,395.0,0.0,...,0.0198,0.0205,0.0190,0.0523,1,0.7758,0.8867,0.0647,29400,36250
724,Johnson University Florida,Kissimmee,FL,0.0,0.0,0.0,1,480.0,470.0,0.0,...,0.0045,0.0045,0.0136,0.1636,1,0.6689,0.7384,0.2185,26300,20199


In [68]:
grouped.head(1)

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,...,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,...,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
43,Prince Institute-Southeast,Elmhurst,IL,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0000,0.0000,0.0000,0.0000,1,0.7857,0.9375,0.6569,PrivacySuppressed,20992
60,University of Alaska Anchorage,Anchorage,AK,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0980,0.0181,0.0457,0.4539,1,0.2385,0.2647,0.4386,42500,19449.5
61,Alaska Bible College,Palmer,AK,0.0,0.0,0.0,1,NaN,NaN,0.0,...,0.0370,0.0000,0.0000,0.1481,1,0.3571,0.2857,0.4286,NaN,PrivacySuppressed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4561,College of the Marshall Islands,Majuro,MH,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0000,0.0056,0.0000,0.2727,1,0.8923,0.0000,0.2310,PrivacySuppressed,PrivacySuppressed
5289,Pacific Islands University,Mangilao,GU,0.0,0.0,0.0,1,NaN,NaN,0.0,...,0.0000,0.0000,0.0000,0.1846,1,0.9730,0.0000,0.2533,PrivacySuppressed,PrivacySuppressed
6439,Touro University Nevada,Henderson,NV,0.0,0.0,0.0,1,NaN,NaN,0.0,...,0.0323,0.0000,0.0645,0.0323,1,0.0000,0.2000,0.4000,NaN,PrivacySuppressed
7404,University of the Virgin Islands-Albert A. Sheen,St. Croix,VI,NaN,NaN,NaN,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,31800,15150


In [75]:
grouped.nth([0, -1])

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,...,0.000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,...,0.000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
43,Prince Institute-Southeast,Elmhurst,IL,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.000,0.0000,0.0000,0.0000,1,0.7857,0.9375,0.6569,PrivacySuppressed,20992
60,University of Alaska Anchorage,Anchorage,AK,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.098,0.0181,0.0457,0.4539,1,0.2385,0.2647,0.4386,42500,19449.5
61,Alaska Bible College,Palmer,AK,0.0,0.0,0.0,1,NaN,NaN,0.0,...,0.037,0.0000,0.0000,0.1481,1,0.3571,0.2857,0.4286,NaN,PrivacySuppressed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7519,Strayer University-Charleston Campus,North Charleston,SC,NaN,NaN,NaN,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,49200,36173.5
7531,Rasmussen College - Overland Park,Overland Park,KS,NaN,NaN,NaN,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,21163
7532,National Personal Training Institute of Cleveland,Highland Heights,OH,NaN,NaN,NaN,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,6333
7533,Bay Area Medical Academy - San Jose Satellite ...,San Jose,CA,NaN,NaN,NaN,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,PrivacySuppressed


In [80]:
grouped = college.set_index('INSTNM').groupby('STABBR')
grouped.ngroups, college['STABBR'].nunique()

(59, 59)

In [81]:
def check_minor(df, threshold=0.5):
    minor_pct = 1 - df['UGDS_WHITE']
    total_minor = (df['UGDS'] * minor_pct).sum()
    total_ugds = df['UGDS'].sum()
    total_minor_pct = total_minor / total_ugds
    return total_minor_pct > threshold

In [82]:
col_filtered = grouped.filter(check_minor, threshold=0.5)

In [83]:
col_filtered

,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
INSTNM,,,,,,,,,,,,,,,,,,,,,
Everest College-Phoenix,Phoenix,AZ,0.0,0.0,0.0,1,NaN,NaN,0.0,4102.0,...,0.0373,0.0,0.1026,0.4749,0,0.8291,0.7151,0.6700,28600,9500
Collins College,Phoenix,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,83.0,...,0.0241,0.0,0.3855,0.3373,0,0.7205,0.8228,0.4764,25700,47000
Empire Beauty School-Paradise Valley,Phoenix,AZ,0.0,0.0,0.0,1,NaN,NaN,0.0,25.0,...,0.0400,0.0,0.0000,0.1600,0,0.6349,0.5873,0.4651,17800,9588
Empire Beauty School-Tucson,Tucson,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,126.0,...,0.0000,0.0,0.0079,0.2222,1,0.7962,0.6615,0.4229,18200,9833
Thunderbird School of Global Management,Glendale,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,1.0,...,0.0000,0.0,0.0000,1.0000,0,0.0000,0.0000,0.0000,118900,PrivacySuppressed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WestMed College - Merced,Merced,CA,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,15623.5
Vantage College,El Paso,TX,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,9500
SAE Institute of Technology San Francisco,Emeryville,CA,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,9500


In [84]:
wloss = pd.read_csv(p1 / 'weight_loss.csv')
wloss

,Name,Month,Week,Weight
0,Bob,Jan,Week 1,291
1,Amy,Jan,Week 1,197
2,Bob,Jan,Week 2,288
3,Amy,Jan,Week 2,189
4,Bob,Jan,Week 3,283
5,Amy,Jan,Week 3,189
6,Bob,Jan,Week 4,283
7,Amy,Jan,Week 4,190
8,Bob,Feb,Week 1,283
9,Amy,Feb,Week 1,190


In [85]:
def pct_loss(s):
    return round(((s - s.iloc[0]) / s.iloc[0]) * 100, 2)

In [87]:
wloss_trans = (wloss
 .groupby(['Name', 'Month'])['Weight']
 .transform(pct_loss)
)
wloss_trans

0     0.00
1     0.00
2    -1.03
3    -4.06
4    -2.75
5    -4.06
6    -2.75
7    -3.55
8     0.00
9     0.00
10   -2.83
11   -3.16
12   -5.30
13   -6.84
14   -5.30
15   -8.95
16    0.00
17    0.00
18    1.12
19    0.00
20   -1.12
21   -1.73
22   -2.61
23   -1.73
24    0.00
25    0.00
26   -1.15
27   -3.53
28   -3.07
29   -3.53
30   -4.21
31   -5.29
Name: Weight, dtype: float64

In [98]:
(wloss
 .assign(pct_loss=wloss_trans)
 .query("Week == 'Week 4'")
 .pivot(index='Month', columns='Name', values='pct_loss')
 .assign(Winner=lambda df: np.where(df.Amy < df.Bob, 'Amy', 'Bob'))
 .reindex(['Jan', 'Feb', 'Mar', 'Apr'])
)

Name,Amy,Bob,Winner
Month,,,
Jan,-3.55,-2.75,Amy
Feb,-8.95,-5.30,Amy
Mar,-1.73,-2.61,Bob
Apr,-5.29,-4.21,Amy


In [100]:
subset = ['UGDS', 'SATMTMID', 'SATVRMID']
col2 = college.dropna(subset=subset)
col2.shape

(1184, 27)

In [101]:
def wma(df):
    wm = df['UGDS'] * df['SATMTMID']
    return int(wm.sum() / df['UGDS'].sum())

In [102]:
col2.groupby('STABBR').apply(wma)

STABBR
AK    503
AL    536
AR    529
AZ    569
CA    564
CO    553
CT    545
DC    621
DE    569
FL    565
GA    540
HI    534
IA    577
ID    509
IL    594
IN    546
KS    491
KY    525
LA    549
MA    597
MD    572
ME    524
MI    586
MN    598
MO    576
MS    527
MT    551
NC    552
ND    546
NE    567
NH    561
NJ    554
NM    529
NV    516
NY    578
OH    569
OK    557
OR    540
PA    553
PR    571
RI    567
SC    549
SD    528
TN    544
TX    548
UT    577
VA    550
VI    390
VT    566
WA    555
WI    593
WV    500
WY    540
dtype: int64

In [105]:
def wa(df):
    wm = df['UGDS'] * df['SATMTMID']
    wv = df['UGDS'] * df['SATVRMID']
    wm_avg = int(wm.sum() / df['UGDS'].sum())
    wv_avg = int(wv.sum() / df['UGDS'].sum())
    data = {'math_weighted':wm_avg, 'verbal_weighted':wv_avg,
            'math_avg':int(df['SATMTMID'].mean()), 'verbal_avg':int(df['SATVRMID'].mean()),
            'count':int(len(df))}
    return pd.Series(data)

In [106]:
col2.groupby('STABBR').apply(wa)

,math_weighted,verbal_weighted,math_avg,verbal_avg,count
STABBR,,,,,
AK,503,555,503,555,1
AL,536,533,504,508,21
AR,529,504,515,491,16
AZ,569,557,536,538,6
CA,564,539,562,549,72
CO,553,547,540,537,14
CT,545,533,522,517,14
DC,621,623,588,589,6
DE,569,553,495,486,3


In [109]:
bins = [-np.inf, 200, 500, 1000, 2000, np.inf]
cuts = pd.cut(flights['DIST'], bins=bins, labels=['U200', '200-500', '500-1000', '1000-2000', 'O2000'])
cuts

0         500-1000
1        1000-2000
2         500-1000
3        1000-2000
4        1000-2000
           ...    
58487    1000-2000
58488      200-500
58489      200-500
58490     500-1000
58491     500-1000
Name: DIST, Length: 58492, dtype: category
Categories (5, object): ['U200' < '200-500' < '500-1000' < '1000-2000' < 'O2000']

In [111]:
(flights
 .groupby(cuts, observed=True)['AIRLINE']
 .value_counts(normalize=True)
 .round(3)
)

DIST       AIRLINE
U200       OO         0.326
           EV         0.289
           MQ         0.211
           DL         0.086
           AA         0.052
           UA         0.027
           WN         0.009
200-500    WN         0.194
           DL         0.189
           OO         0.159
           EV         0.156
           MQ         0.100
           AA         0.071
           UA         0.062
           VX         0.028
           US         0.016
           NK         0.012
           B6         0.007
           F9         0.005
           AS         0.001
500-1000   DL         0.206
           AA         0.144
           WN         0.138
           UA         0.131
           OO         0.106
           EV         0.101
           MQ         0.051
           F9         0.038
           NK         0.030
           US         0.025
           AS         0.023
           VX         0.004
           B6         0.003
1000-2000  AA         0.264
           UA         0.199
 

In [117]:
(flights
 .groupby(cuts, observed=True)['AIRLINE']
 .size()
 .round(3)
)

DIST
U200          3719
200-500      15874
500-1000     20659
1000-2000    14186
O2000         4054
Name: AIRLINE, dtype: int64

In [121]:
data_sorted = np.sort(flights[['ORG_AIR', 'DEST_AIR']], axis=1)
data_sorted[:10]

array([['LAX', 'SLC'],
       ['DEN', 'IAD'],
       ['DFW', 'VPS'],
       ['DCA', 'DFW'],
       ['LAX', 'MCI'],
       ['IAH', 'SAN'],
       ['DFW', 'MSY'],
       ['PHX', 'SFO'],
       ['ORD', 'STL'],
       ['IAH', 'SJC']], dtype=object)

In [124]:
sorted_air = pd.DataFrame(data_sorted, columns=['AIR1', 'AIR2'])
sorted_air

,AIR1,AIR2
0,LAX,SLC
1,DEN,IAD
2,DFW,VPS
3,DCA,DFW
4,LAX,MCI
...,...,...
58487,DFW,SFO
58488,LAS,SFO
58489,SBA,SFO
58490,ATL,MSP


In [125]:
(pd.concat([flights, sorted_air], axis=1)
 .groupby(['AIR1', 'AIR2'])
 .size()
)

AIR1  AIR2
ABE   ATL      31
      ORD      24
ABI   DFW      74
ABQ   ATL      16
      DEN      46
             ... 
SFO   SNA     122
      STL      20
      SUN      10
      TUS      20
      XNA       2
Length: 1085, dtype: int64

In [128]:
(pd.concat([flights, sorted_air], axis=1)
 .groupby(['AIR1', 'AIR2'])
 .size()
 .loc[('ATL', 'IAH')]
)

269

In [130]:
ser = pd.Series([0, 1, 1, 0, 1, 1, 1, 0])
ser

0    0
1    1
2    1
3    0
4    1
5    1
6    1
7    0
dtype: int64

In [131]:
s1 = ser.cumsum()
s1

0    0
1    1
2    2
3    2
4    3
5    4
6    5
7    5
dtype: int64

In [133]:
s2 = s1 * ser
s2

0    0
1    1
2    2
3    0
4    3
5    4
6    5
7    0
dtype: int64

In [139]:
s2.diff().where(lambda x: x < 0).ffill().add(s1, fill_value=0).astype('int')

0    0
1    1
2    2
3    0
4    1
5    2
6    3
7    0
dtype: int32

In [141]:
s3 = s2.diff()
s3

0    NaN
1    1.0
2    1.0
3   -2.0
4    3.0
5    1.0
6    1.0
7   -5.0
dtype: float64

In [142]:
s4 = s3.where(lambda x: x < 0)
s4

0    NaN
1    NaN
2    NaN
3   -2.0
4    NaN
5    NaN
6    NaN
7   -5.0
dtype: float64

In [143]:
s5 = s4.ffill()
s5

0    NaN
1    NaN
2    NaN
3   -2.0
4   -2.0
5   -2.0
6   -2.0
7   -5.0
dtype: float64

In [144]:
s6 = s5.add(s1, fill_value=0).astype('int')
s6

0    0
1    1
2    2
3    0
4    1
5    2
6    3
7    0
dtype: int32

In [149]:
def max_streak(ser):
    s1 = ser.cumsum()
    s2 = s1.mul(ser)
    s3 = s2.diff()
    s4 = s3.where(lambda x: x < 0)
    s5 = s4.ffill()
    s6 = s5.add(s1, fill_value=0).astype('int')
    return s6.max()

In [150]:
(flights
 .assign(ON_TIME=flights['ARR_DELAY'].lt(15).astype('int'))
 .sort_values(['MONTH', 'DAY', 'SCHED_DEP'])
 .groupby(['AIRLINE', 'ORG_AIR'])['ON_TIME']
 .agg(['size', 'mean', max_streak])
 .round(2)
)

size  mean  max_streak
AIRLINE ORG_AIR                        
AA      ATL       233  0.82          15
        DEN       219  0.74          17
        DFW      4006  0.78          64
        IAH       196  0.80          24
        LAS       374  0.79          29
...               ...   ...         ...
WN      LAS      2031  0.77          39
        LAX      1135  0.70          23
        MSP       237  0.84          32
        PHX      1724  0.77          33
        SFO       445  0.76          17

[114 rows x 3 columns]